# Introducción
En este notebook, se implementan y analizan diferentes técnicas de embeddings de texto utilizando reseñas del producto ALEXA de Amazon, entonces como todo proyecto, se tiene que hacer la lectura del archivo así como su posterior, limpieza y luego usar las diferentes técnicas de analisis de texto que se conocen como lo es el TF-IDF, Bag-Of-Words y Word2Vec. Luego de hacer los respectivos analisis con sus diferentes puntos de vista lo que se tiene que hacer es usar una herramienta de ML (en este caso Random Forest) para poder clasificar los sentimientos del dataset y poder ver en que todo fue la reseña, si buena o mala
# Objetivo
- Comparar las precisiones de los diferentes analizadores de texto (TF-IDF, Bag-of-Words y Word2Vec).
- Crear un modelo de ML para hacer predicciones sobre los sentimientos de cada reseña
- Entender en que caso es mejor cada uno y como funciona toda la parte del Text Embeddings
- Analizar y discutir los resultados obtenidos de cada método y su aplicabilidad en diferentes contextos.

In [29]:
# Importación de la lectura de archivos
import pandas as pd

In [30]:
# Lectura del archivo y limitación en cuanto a las columnas para que el código se ejecute adecuadamente
df = pd.read_csv('Reviews.csv')
df.columns = ['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator', 'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text']
df = df.head(100000)
# Muestra del dataset ya completo y leido
print(df.head())
print(df.shape)

   Id   ProductId          UserId                      ProfileName  \
0   1  B001E4KFG0  A3SGXH7AUHU8GW                       delmartian   
1   2  B00813GRG4  A1D87F6ZCVE5NK                           dll pa   
2   3  B000LQOCH0   ABXLMWJIXXAIN  Natalia Corres "Natalia Corres"   
3   4  B000UA0QIQ  A395BORC6FGVXV                             Karl   
4   5  B006K2ZZ7K  A1UQRSCLF8GW1T    Michael D. Bigham "M. Wassir"   

   HelpfulnessNumerator  HelpfulnessDenominator  Score        Time  \
0                     1                       1      5  1303862400   
1                     0                       0      1  1346976000   
2                     1                       1      4  1219017600   
3                     3                       3      2  1307923200   
4                     0                       0      5  1350777600   

                 Summary                                               Text  
0  Good Quality Dog Food  I have bought several of the Vitality canned d...  
1 

In [31]:
# Declaración del dataset, para poder usar el TF-IDF y Bag-Of-Word
df_review = df[['Score', 'Text']]
# Declaración del dataset para el Word2Vec
df_review_word2vec = df[['Score', 'Text']]
# Limpieza de datos del Score, para cambiarlo a enteros y ajustar las calificaciones de 4 o 5, es bueno y 3 para abajo es malo
df_review['Score'] = df_review['Score'].astype(int)
df_review['Score'] = df_review['Score'].apply(lambda x: 1 if x > 3 else 0)
# Mostrar los datos
df_review.head()

<ipython-input-31-0a0a6c60bbf8>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_review['Score'] = df_review['Score'].astype(int)
<ipython-input-31-0a0a6c60bbf8>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_review['Score'] = df_review['Score'].apply(lambda x: 1 if x > 3 else 0)


,Score,Text
0,1,I have bought several of the Vitality canned d...
1,0,Product arrived labeled as Jumbo Salted Peanut...
2,1,This is a confection that has been around a fe...
3,0,If you are looking for the secret ingredient i...
4,1,Great taffy at a great price. There was a wid...


# TF-IDF

In [32]:
# Implementación de las librerías necesarias para evaluar el modelo con TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Asignar lo que son los features y labels en el modelo del Random Forest
X = df_review['Text']
y = df_review['Score']

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Usar la Vectorización TF-IDF para prueba del primer modelo
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Usar el modelo del Random Forest para clasificar lo que son los comentarios entre buenos y malos
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train_tfidf, y_train)

# Evaluación del modelo haciendo predicciones y viendo su resultado en la precisión
y_pred = rf_model.predict(X_test_tfidf)

# Ver el rendimiento del modelo
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred, target_names=['Malo', 'Bueno'])

# Mostar los resultados del modelo tanto en su precisión como en los demás scores
print("Precisión del modelo:", accuracy)
print("\nReporte de clasificación:\n", classification_rep)

Precisión del modelo: 0.87155

Reporte de clasificación:
               precision    recall  f1-score   support

        Malo       0.89      0.50      0.64      4595
       Bueno       0.87      0.98      0.92     15405

    accuracy                           0.87     20000
   macro avg       0.88      0.74      0.78     20000
weighted avg       0.87      0.87      0.86     20000



# Bag-of-Words(BOW)

In [33]:
# Implementación de las librerías para el uso del Bag-Of-Words
from sklearn.feature_extraction.text import CountVectorizer

# Usar el modelo de Vectorización Bag-of-Words para la evaluación del modelo
vectorizer = CountVectorizer(max_features=5000, stop_words='english')
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)

# Usar el modelo del Random Forest para clasificar lo que son los comentarios entre buenos y malos
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train_bow, y_train)

# Evaluación del modelo haciendo predicciones y viendo su resultado en la precisión
y_pred = rf_model.predict(X_test_bow)

# Ver el rendimiento del modelo
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred, target_names=['Malo', 'Bueno'])

# Mostar los resultados del modelo tanto en su precisión como en los demás scores
print("Precisión del modelo:", accuracy)
print("\nReporte de clasificación:\n", classification_rep)

Precisión del modelo: 0.8716

Reporte de clasificación:
               precision    recall  f1-score   support

        Malo       0.87      0.52      0.65      4595
       Bueno       0.87      0.98      0.92     15405

    accuracy                           0.87     20000
   macro avg       0.87      0.75      0.79     20000
weighted avg       0.87      0.87      0.86     20000



# Word2Vec

In [34]:
pip install gensim

In [35]:
# Implementación de la librería para poder usar el Word2Vec
from gensim.models import Word2Vec
import numpy as np

# Preprocesar el texto para Word2Vec, convertir los datos a minuscula y dividirlo por espacios
X_train_word2vec = X_train.str.lower().str.split()
X_test_word2vec = X_test.str.lower().str.split()

# Se utiliza el modelo Word2Vec para hacer un analisis de las palabras
model_w2v = Word2Vec(sentences=X_train_word2vec, vector_size=100, window=5, min_count=1, workers=4)

# Función para obtener el vector promedio de una reseña
def get_vector(text):
    vectors = []
    for word in text:
        if word in model_w2v.wv:
            vectors.append(model_w2v.wv[word])
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model_w2v.vector_size)

# Obtener los vectores para cada reseña
X_train_w2v = np.array([get_vector(text) for text in X_train_word2vec])
X_test_w2v = np.array([get_vector(text) for text in X_test_word2vec])
y_train_w2v = y_train.values
y_test_w2v = y_test.values

# Usar el modelo del Random Forest para clasificar lo que son los comentarios entre buenos y malos
rf_model_w2v = RandomForestClassifier(random_state=42)
rf_model_w2v.fit(X_train_w2v, y_train_w2v)

# Evaluación del modelo haciendo predicciones y viendo su resultado en la precisión
y_pred_w2v = rf_model_w2v.predict(X_test_w2v)

# Ver el rendimiento del modeloo
accuracy_w2v = accuracy_score(y_test_w2v, y_pred_w2v)
classification_rep_w2v = classification_report(y_test_w2v, y_pred_w2v, target_names=['Malo', 'Bueno'])

# Mostar los resultados del modelo tanto en su precisión como en los demás scores
print("--------------------------------------------------------")
print("Precisión del modelo:", accuracy)
print("\nReporte de clasificación:\n", classification_rep)

--------------------------------------------------------
Precisión del modelo: 0.8716

Reporte de clasificación:
               precision    recall  f1-score   support

        Malo       0.87      0.52      0.65      4595
       Bueno       0.87      0.98      0.92     15405

    accuracy                           0.87     20000
   macro avg       0.87      0.75      0.79     20000
weighted avg       0.87      0.87      0.86     20000



# Conclusión
En este caso se puede analizar que los resultados en los diferentes usos de los analizadores de texto son bastante parecidos con diferencias muy mínimas.
- TF-IDF: Precisión 87.155%
- Bag-Of-Words: Precisión 87.16%
- Word2Vec: Precisión 87.16%

Esto lo lograron incluso limitando el dataset ya que se se hubiera analizado los 568456 registros que eran puede que esto pueda llegar a variar. Se puede ver que practicamente sacaron los mismos resultados, usando los mismo datos de entrenamiento y de prueba. Esto puede deberse para lo que es el modelo de ML, ya que usamos el Random Forest lo que los puede llegar a hacer muy eficientes, inclusive si contamos con las diferentes formas en que se analizaron los textos, realmente son resultados muy aceptables que no tienden al overfiting, pero si desconocemos como realizo el árbol para cada uno de los casos, puede que sean arboles demasiado complejos que por eso saca la presición que saca.

Independiemnte del caso en está ocasión por la naturaleza del problema y tal vez la limitación del dataset, se puede ver que los diferentes analizadores de texto sacaron, una precisión igual, y buena en este caso.

Independientemente del caso, por la naturaleza del problema y la limitación del dataset, se observa que los diferentes analizadores de texto obtuvieron una precisión similar y aceptable. Además, dado que las reseñas se relacionan con el mismo producto, puede haber una falta de variedad en las descripciones, lo que contribuyó a resultados similares.